In [26]:
# Konfigurasi Spark
import os
import sys

# 1. Mengeset variabel yang menyimpan lokasi di mana Spark diinstal
spark_path = "D:/spark"

# 2. Menentukan environment variable SPARK_HOME
os.environ['SPARK_HOME'] = spark_path

# 3. Simpan lokasi winutils.exe sebagai environment variable HADOOP_HOME
os.environ['HADOOP_HOME'] = spark_path

# 4. Lokasi Python yang dijalankan --> punya Anaconda
#    Apabila Python yang diinstall hanya Anaconda, maka tidak perlu menjalankan baris ini.
os.environ['PYSPARK_PYTHON'] = sys.executable

# 5. Konfigurasi path library PySpark
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

# 6. Import library Spark
#    Dua library yang WAJIB di-import adalah **SparkContext** dan **SparkConf**.
from pyspark import SparkContext
from pyspark import SparkConf

# Setting konfigurasi (opsional)
conf = SparkConf()
conf.set("spark.executor.memory", "2g")
conf.set("spark.cores.max", "4")

sc = SparkContext.getOrCreate()
#    Apabila berhasil, maka ketika sc di-print akan mengeluarkan nilai <pyspark.context.SparkContext object>
print sc

In [57]:
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel

dataPath ="exercise-4/complete.csv"
datacomplete=sc.textFile(dataPath)

In [58]:
print datacomplete

exercise-4/complete.csv MapPartitionsRDD[39] at textFile at <unknown>:0


In [59]:
datacomplete.collect()

[u'datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude',
 u'5/22/1974 5:30,mescalero indian reservation,nm,,rectangle,180,two hours,Huge rectangular object emmitting intense white light across roadway with extremely loud roar,4/18/2012,-82.8628,-176.658',
 u'3/15/2014 21:50,,,,,0,,folsom lake/eldorado hill,My son and I were going for a late night Taco Bell run on the evening of Saturday the 15th at 9:50 pm.&#160; As we drove East on Central Ave,-82.8628,-176.658',
 u'2/7/2014 18:30,,tx,,,0,disk,seconds,Saucer-like craft with a strip of red light moving back and forth&#44 hovering above a gas well site.,-82.8628,-176.658',
 u'3/1/2014 6:00,,pa,,,0,teardrop,15 minutes,I saw 3 lights circling in sky in PA on 15 north route as soon as I270 North ends. I saw it for 15/20 min and then it disappeared.,-82.8628,-176.658',
 u'7/20/1980 13:00,,fl,,,0,triangle,~10 minutes,Large Triangular craft going East over the Atlantic Ocean .,-82.

In [43]:
datacomplete.count()

88876

In [80]:
dataheader = datacomplete.filter(lambda x:"datetime,city" not in x)
datasplit = dataheader.map(lambda x:x.split(','))
datalatlng = datasplit.map(lambda x:x[9:])
datarr = datalatlng.map(lambda line: array([x for x in line]))
data = datarr.map(lambda line: ' '.join(line[0:2]))

In [83]:
parsedData = data.map(lambda line: array([float(x.strip()) for x in line.split(' ')]))

In [84]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel

clusters = KMeans.train(parsedData, 3, maxIterations=1000, initializationMode="random")

In [85]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 1128077.43948


In [86]:
clusterData = parsedData.map(clusters.predict)
X = parsedData.collect()
label = clusterData.collect()

In [88]:
import numpy as np
import matplotlib.pyplot as plt


for point in range(len(label)):
    if (label[point]==0):
        color = "r";
    elif(label[point]==1) :
        color = "g";
    else :
        color = "b";
    lines = plt.plot(X[point][0], X[point][1], 'ro')
    plt.setp(lines, color=color, linewidth=2.0)

KeyboardInterrupt: 

In [ ]:
plt.show()